<h2> Purpose of this step is to filter out the non-protein coding genes from GTEX </h2>
<p> Input: </p>
<ol>
<li> PCG symbol from gene genenmes after processing </li>
<li> GTEX matrix contain at total of 56200 genes and 17382 samples </li>
</ol>
<p> Output: </p>
<ol>
<li> A matrix containing on the ENSMBL description and only the genes that were in the intersection of <br>
    the protein coding genes and GTEx matrix. </li>
<li> A dictionary from ENSMBL to description </li>
</ol>

<p> Other features </p>
<ol>
<li>
Log transform to matrix
</li>
</ol>

In [1]:
import pandas as pd
import os 
from pathlib import Path
import copy
import math
import numpy as np

In [2]:
parent_path = os.getcwd()
print(os.getcwd())

/Users/sussman/Desktop/Dynamic19Cohort1


In [3]:
source_GTEX = 'GTEX_raw_data'
os.chdir(source_GTEX)

In [4]:
try1 = pd.read_csv('GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct', sep='\t',skiprows=2)

In [5]:
display(try1.head())

Name  Description  GTEX-1117F-0226-SM-5GZZ7  \
0  ENSG00000223972.5      DDX11L1                   0.00000   
1  ENSG00000227232.5       WASH7P                   8.76400   
2  ENSG00000278267.1    MIR6859-1                   0.00000   
3  ENSG00000243485.5  MIR1302-2HG                   0.07187   
4  ENSG00000237613.2      FAM138A                   0.00000   

   GTEX-1117F-0426-SM-5EGHI  GTEX-1117F-0526-SM-5EGHJ  \
0                     0.000                     0.000   
1                     3.861                     7.349   
2                     0.000                     1.004   
3                     0.000                     0.000   
4                     0.000                     0.000   

   GTEX-1117F-0626-SM-5N9CS  GTEX-1117F-0726-SM-5GIEN  \
0                   0.00000                     0.000   
1                  11.07000                     3.306   
2                   0.00000                     0.000   
3                   0.06761                     0.000   
4                   0.00000                     0.000   

   GTEX-1117F-1326-SM-5EGHH  GTEX-1117F-2426-SM-5EGGH  \
0                     0.000                      0.00   
1                     5.389                     11.99   
2                     0.000                      0.00   
3                     0.000                      0.00   
4                     0.000                      0.00   

   GTEX-1117F-2526-SM-5GZY6  ...  GTEX-ZZPU-1126-SM-5N9CW  \
0                   0.00000  ...                  0.00000   
1                  16.95000  ...                  1.60600   
2                   0.00000  ...                  0.00000   
3                   0.00000  ...                  0.00000   
4                   0.03904  ...                  0.02429   

   GTEX-ZZPU-1226-SM-5N9CK  GTEX-ZZPU-1326-SM-5GZWS  GTEX-ZZPU-1426-SM-5GZZ6  \
0                  0.03629                  0.00000                     0.00   
1                  2.26800                  5.38600                     2.31   
2                  0.00000                  0.00000                     0.00   
3                  0.00000                  0.06073                     0.00   
4                  0.00000                  0.00000                     0.00   

   GTEX-ZZPU-1826-SM-5E43L  GTEX-ZZPU-2126-SM-5EGIU  GTEX-ZZPU-2226-SM-5EGIV  \
0                  0.00000                   0.0000                    0.000   
1                  2.45600                   4.0230                    1.922   
2                  0.00000                   0.0000                    0.000   
3                  0.08464                   0.1435                    0.000   
4                  0.00000                   0.0000                    0.000   

   GTEX-ZZPU-2426-SM-5E44I  GTEX-ZZPU-2626-SM-5E45Y  GTEX-ZZPU-2726-SM-5NQ8O  
0                  0.00000                  0.01965                  0.02522  
1                  2.85700                  0.86960                  2.16700  
2                  0.00000                  0.00000                  0.00000  
3                  0.05216                  0.00000                  0.00000  
4                  0.00000                  0.00000                  0.00000  

[5 rows x 17384 columns]

In [6]:
try1.shape

(56200, 17384)

In [7]:
os.chdir(parent_path)

In [8]:
GTEX_intermediate = 'GTEX_intermediate_data'

In [9]:
os.chdir(GTEX_intermediate)

In [10]:
pcg = pd.read_csv('pcgSymbols.csv',index_col=0)

In [11]:
display(pcg.head())

PCG Symbols
0        A1BG
1        A1CF
2         A2M
3       A2ML1
4     A3GALT2

In [12]:
print(pcg.shape)

(19208, 1)


In [13]:
PCG_list = pcg['PCG Symbols'].tolist()

In [14]:
print(len(PCG_list))
print(type(PCG_list))

19208
<class 'list'>


In [15]:
#df[df['A'] in list_of_values]
matrix_pcg_df = try1[try1['Description'].isin(PCG_list)]

In [16]:
display(matrix_pcg_df.head())

Name Description  GTEX-1117F-0226-SM-5GZZ7  \
7    ENSG00000186092.4       OR4F5                   0.00000   
25   ENSG00000278566.1      OR4F29                   0.00000   
39   ENSG00000273547.1      OR4F16                   0.06628   
57  ENSG00000187634.11      SAMD11                   0.35980   
58  ENSG00000188976.10       NOC2L                  90.74000   

    GTEX-1117F-0426-SM-5EGHI  GTEX-1117F-0526-SM-5EGHJ  \
7                    0.10250                   0.07434   
25                   0.05009                   0.00000   
39                   0.00000                   0.00000   
57                   0.47600                   0.27620   
58                  93.56000                  94.87000   

    GTEX-1117F-0626-SM-5N9CS  GTEX-1117F-0726-SM-5GIEN  \
7                       0.00                   0.04233   
25                      0.00                   0.00000   
39                      0.00                   0.04138   
57                     12.03                  11.39000   
58                     67.60                  26.52000   

    GTEX-1117F-1326-SM-5EGHH  GTEX-1117F-2426-SM-5EGGH  \
7                    0.05609                   0.17430   
25                   0.05483                   0.08519   
39                   0.00000                   0.00000   
57                   1.71200                  25.12000   
58                  33.67000                  85.62000   

    GTEX-1117F-2526-SM-5GZY6  ...  GTEX-ZZPU-1126-SM-5N9CW  \
7                    0.00000  ...                  0.12900   
25                   0.05068  ...                  0.00000   
39                   0.00000  ...                  0.03154   
57                   2.09100  ...                  6.65200   
58                  93.06000  ...                 34.88000   

    GTEX-ZZPU-1226-SM-5N9CK  GTEX-ZZPU-1326-SM-5GZWS  GTEX-ZZPU-1426-SM-5GZZ6  \
7                   0.13670                  0.05729                  0.12690   
25                  0.03341                  0.11200                  0.00000   
39                  0.03341                  0.00000                  0.04136   
57                  4.76100                  1.24700                  1.76300   
58                 42.61000                 60.42000                 41.23000   

    GTEX-ZZPU-1826-SM-5E43L  GTEX-ZZPU-2126-SM-5EGIU  GTEX-ZZPU-2226-SM-5EGIV  \
7                   0.11980                  0.06767                  0.11350   
25                  0.11710                  0.00000                  0.05548   
39                  0.07806                  0.19850                  0.16640   
57                  1.02800                  0.80820                  0.45190   
58                 52.21000                 55.73000                 71.21000   

    GTEX-ZZPU-2426-SM-5E44I  GTEX-ZZPU-2626-SM-5E45Y  GTEX-ZZPU-2726-SM-5NQ8O  
7                    0.0000                   0.1480                  0.00000  
25                   0.1924                   0.1447                  0.00000  
39                   0.0000                   0.0000                  0.04644  
57                   6.3460                   1.0510                  0.16390  
58                  58.7600                  75.2200                 39.30000  

[5 rows x 17384 columns]

In [17]:
print(matrix_pcg_df.shape)

(18349, 17384)


In [18]:
#matrix_somekh_df = try1[try1['Description'].isin(somekh_list)]

In [19]:
#print(matrix_somekh_df.shape)

In [20]:
pcg_dictionary = matrix_pcg_df[['Name','Description']]
pcg_dictionary = pcg_dictionary.set_index('Name')

In [21]:
display(pcg_dictionary.head())

Description
Name                          
ENSG00000186092.4        OR4F5
ENSG00000278566.1       OR4F29
ENSG00000273547.1       OR4F16
ENSG00000187634.11      SAMD11
ENSG00000188976.10       NOC2L

In [22]:
pcg_dictionary.to_csv('pcg_dict_name_to_description.csv')

In [23]:
matrix_pcg_df =matrix_pcg_df.drop(['Description'], axis=1)

In [24]:
matrix_pcg_df.set_index('Name',inplace=True)
display(matrix_pcg_df.head())

GTEX-1117F-0226-SM-5GZZ7  GTEX-1117F-0426-SM-5EGHI  \
Name                                                                     
ENSG00000186092.4                    0.00000                   0.10250   
ENSG00000278566.1                    0.00000                   0.05009   
ENSG00000273547.1                    0.06628                   0.00000   
ENSG00000187634.11                   0.35980                   0.47600   
ENSG00000188976.10                  90.74000                  93.56000   

                    GTEX-1117F-0526-SM-5EGHJ  GTEX-1117F-0626-SM-5N9CS  \
Name                                                                     
ENSG00000186092.4                    0.07434                      0.00   
ENSG00000278566.1                    0.00000                      0.00   
ENSG00000273547.1                    0.00000                      0.00   
ENSG00000187634.11                   0.27620                     12.03   
ENSG00000188976.10                  94.87000                     67.60   

                    GTEX-1117F-0726-SM-5GIEN  GTEX-1117F-1326-SM-5EGHH  \
Name                                                                     
ENSG00000186092.4                    0.04233                   0.05609   
ENSG00000278566.1                    0.00000                   0.05483   
ENSG00000273547.1                    0.04138                   0.00000   
ENSG00000187634.11                  11.39000                   1.71200   
ENSG00000188976.10                  26.52000                  33.67000   

                    GTEX-1117F-2426-SM-5EGGH  GTEX-1117F-2526-SM-5GZY6  \
Name                                                                     
ENSG00000186092.4                    0.17430                   0.00000   
ENSG00000278566.1                    0.08519                   0.05068   
ENSG00000273547.1                    0.00000                   0.00000   
ENSG00000187634.11                  25.12000                   2.09100   
ENSG00000188976.10                  85.62000                  93.06000   

                    GTEX-1117F-2826-SM-5GZXL  GTEX-1117F-2926-SM-5GZYI  ...  \
Name                                                                    ...   
ENSG00000186092.4                    0.00000                    0.0591  ...   
ENSG00000278566.1                    0.19860                    0.0000  ...   
ENSG00000273547.1                    0.04965                    0.0000  ...   
ENSG00000187634.11                   0.67390                    0.6117  ...   
ENSG00000188976.10                  64.54000                   80.9500  ...   

                    GTEX-ZZPU-1126-SM-5N9CW  GTEX-ZZPU-1226-SM-5N9CK  \
Name                                                                   
ENSG00000186092.4                   0.12900                  0.13670   
ENSG00000278566.1                   0.00000                  0.03341   
ENSG00000273547.1                   0.03154                  0.03341   
ENSG00000187634.11                  6.65200                  4.76100   
ENSG00000188976.10                 34.88000                 42.61000   

                    GTEX-ZZPU-1326-SM-5GZWS  GTEX-ZZPU-1426-SM-5GZZ6  \
Name                                                                   
ENSG00000186092.4                   0.05729                  0.12690   
ENSG00000278566.1                   0.11200                  0.00000   
ENSG00000273547.1                   0.00000                  0.04136   
ENSG00000187634.11                  1.24700                  1.76300   
ENSG00000188976.10                 60.42000                 41.23000   

                    GTEX-ZZPU-1826-SM-5E43L  GTEX-ZZPU-2126-SM-5EGIU  \
Name                                                                   
ENSG00000186092.4                   0.11980                  0.06767   
ENSG00000278566.1                   0.11710                  0.00000   
ENSG00000273547.1                   0.07806                  0.19850   
ENSG00000187634.11  

In [25]:

matrix_pcg_df = matrix_pcg_df +1
display(matrix_pcg_df.head())


GTEX-1117F-0226-SM-5GZZ7  GTEX-1117F-0426-SM-5EGHI  \
Name                                                                     
ENSG00000186092.4                    1.00000                   1.10250   
ENSG00000278566.1                    1.00000                   1.05009   
ENSG00000273547.1                    1.06628                   1.00000   
ENSG00000187634.11                   1.35980                   1.47600   
ENSG00000188976.10                  91.74000                  94.56000   

                    GTEX-1117F-0526-SM-5EGHJ  GTEX-1117F-0626-SM-5N9CS  \
Name                                                                     
ENSG00000186092.4                    1.07434                      1.00   
ENSG00000278566.1                    1.00000                      1.00   
ENSG00000273547.1                    1.00000                      1.00   
ENSG00000187634.11                   1.27620                     13.03   
ENSG00000188976.10                  95.87000                     68.60   

                    GTEX-1117F-0726-SM-5GIEN  GTEX-1117F-1326-SM-5EGHH  \
Name                                                                     
ENSG00000186092.4                    1.04233                   1.05609   
ENSG00000278566.1                    1.00000                   1.05483   
ENSG00000273547.1                    1.04138                   1.00000   
ENSG00000187634.11                  12.39000                   2.71200   
ENSG00000188976.10                  27.52000                  34.67000   

                    GTEX-1117F-2426-SM-5EGGH  GTEX-1117F-2526-SM-5GZY6  \
Name                                                                     
ENSG00000186092.4                    1.17430                   1.00000   
ENSG00000278566.1                    1.08519                   1.05068   
ENSG00000273547.1                    1.00000                   1.00000   
ENSG00000187634.11                  26.12000                   3.09100   
ENSG00000188976.10                  86.62000                  94.06000   

                    GTEX-1117F-2826-SM-5GZXL  GTEX-1117F-2926-SM-5GZYI  ...  \
Name                                                                    ...   
ENSG00000186092.4                    1.00000                    1.0591  ...   
ENSG00000278566.1                    1.19860                    1.0000  ...   
ENSG00000273547.1                    1.04965                    1.0000  ...   
ENSG00000187634.11                   1.67390                    1.6117  ...   
ENSG00000188976.10                  65.54000                   81.9500  ...   

                    GTEX-ZZPU-1126-SM-5N9CW  GTEX-ZZPU-1226-SM-5N9CK  \
Name                                                                   
ENSG00000186092.4                   1.12900                  1.13670   
ENSG00000278566.1                   1.00000                  1.03341   
ENSG00000273547.1                   1.03154                  1.03341   
ENSG00000187634.11                  7.65200                  5.76100   
ENSG00000188976.10                 35.88000                 43.61000   

                    GTEX-ZZPU-1326-SM-5GZWS  GTEX-ZZPU-1426-SM-5GZZ6  \
Name                                                                   
ENSG00000186092.4                   1.05729                  1.12690   
ENSG00000278566.1                   1.11200                  1.00000   
ENSG00000273547.1                   1.00000                  1.04136   
ENSG00000187634.11                  2.24700                  2.76300   
ENSG00000188976.10                 61.42000                 42.23000   

                    GTEX-ZZPU-1826-SM-5E43L  GTEX-ZZPU-2126-SM-5EGIU  \
Name                                                                   
ENSG00000186092.4                   1.11980                  1.06767   
ENSG00000278566.1                   1.11710                  1.00000   
ENSG00000273547.1                   1.07806                  1.19850   
ENSG00000187634.11  

In [26]:
matrix_pcg_df = np.log2(matrix_pcg_df)

In [27]:
display(matrix_pcg_df.head())

GTEX-1117F-0226-SM-5GZZ7  GTEX-1117F-0426-SM-5EGHI  \
Name                                                                     
ENSG00000186092.4                   0.000000                  0.140779   
ENSG00000278566.1                   0.000000                  0.070513   
ENSG00000273547.1                   0.092586                  0.000000   
ENSG00000187634.11                  0.443394                  0.561693   
ENSG00000188976.10                  6.519479                  6.563158   

                    GTEX-1117F-0526-SM-5EGHJ  GTEX-1117F-0626-SM-5N9CS  \
Name                                                                     
ENSG00000186092.4                   0.103451                  0.000000   
ENSG00000278566.1                   0.000000                  0.000000   
ENSG00000273547.1                   0.000000                  0.000000   
ENSG00000187634.11                  0.351854                  3.703765   
ENSG00000188976.10                  6.583008                  6.100137   

                    GTEX-1117F-0726-SM-5GIEN  GTEX-1117F-1326-SM-5EGHH  \
Name                                                                     
ENSG00000186092.4                   0.059812                  0.078733   
ENSG00000278566.1                   0.000000                  0.077011   
ENSG00000273547.1                   0.058497                  0.000000   
ENSG00000187634.11                  3.631104                  1.439357   
ENSG00000188976.10                  4.782409                  5.115616   

                    GTEX-1117F-2426-SM-5EGGH  GTEX-1117F-2526-SM-5GZY6  \
Name                                                                     
ENSG00000186092.4                   0.231801                  0.000000   
ENSG00000278566.1                   0.117948                  0.071323   
ENSG00000273547.1                   0.000000                  0.000000   
ENSG00000187634.11                  4.707083                  1.628074   
ENSG00000188976.10                  6.436628                  6.555509   

                    GTEX-1117F-2826-SM-5GZXL  GTEX-1117F-2926-SM-5GZYI  ...  \
Name                                                                    ...   
ENSG00000186092.4                   0.000000                  0.082839  ...   
ENSG00000278566.1                   0.261350                  0.000000  ...   
ENSG00000273547.1                   0.069908                  0.000000  ...   
ENSG00000187634.11                  0.743213                  0.688583  ...   
ENSG00000188976.10                  6.034304                  6.356672  ...   

                    GTEX-ZZPU-1126-SM-5N9CW  GTEX-ZZPU-1226-SM-5N9CK  \
Name                                                                   
ENSG00000186092.4                  0.175045                 0.184852   
ENSG00000278566.1                  0.000000                 0.047413   
ENSG00000273547.1                  0.044800                 0.047413   
ENSG00000187634.11                 2.935837                 2.526319   
ENSG00000188976.10                 5.165108                 5.446587   

                    GTEX-ZZPU-1326-SM-5GZWS  GTEX-ZZPU-1426-SM-5GZZ6  \
Name                                                                   
ENSG00000186092.4                  0.080371                 0.172359   
ENSG00000278566.1                  0.153157                 0.000000   
ENSG00000273547.1                  0.000000                 0.058469   
ENSG00000187634.11                 1.168000                 1.466236   
ENSG00000188976.10                 5.940637                 5.400196   

                    GTEX-ZZPU-1826-SM-5E43L  GTEX-ZZPU-2126-SM-5EGIU  \
Name                                                                   
ENSG00000186092.4                  0.163241                 0.094466   
ENSG00000278566.1                  0.159758                 0.000000   
ENSG00000273547.1                  0.108437                 0.261230   
ENSG00000187634.11  

In [28]:
print(matrix_pcg_df.shape)

(18349, 17382)


In [29]:
matrix_pcg_df.to_csv('matrix_pcg.csv')

In [30]:
print("done!")

done!
